In [3]:
import pandas as pd
import pickle
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

### Getting Song attributes from Spotify using Spotify API

In [4]:
cid = ""
secret = ""

#initiating Spotify client
client_credentials_manager = SpotifyClientCredentials(client_id = cid, client_secret = secret)
sp = spotipy.Spotify(client_credentials_manager = client_credentials_manager)

In [5]:
songs = pd.read_pickle("songsWithSpotifyID.pkl")

In [6]:
songs.head()

,Title,Artist,SpotifyID
0,Fine Line,Harry Styles,6VzcQuzTNTMFnJ6rBSaLH9
1,Hollywood's Bleeding,Post Malone,7sWRlDoTDX8geTR8zzr2vt
2,Lover,Taylor Swift,1dGr1c8CrMLDpV6mPbImSI
3,Rudolph The Red-Nosed Reindeer,Burl Ives,47otoIkhx3fkdivEXL5OB6
4,What You See Is What You Get,Luke Combs,273TiTHLlHSRe5zrzs7wvD


In [7]:
#initialize the columns
songs["Tempo"] = None
songs["Loudness"] = None
songs["Key"] = None
songs["isExplicit"] = None
songs["Duration"] = None
songs["ReleaseDate"] = None

In [8]:
#getExplicit function that takes in a songID and returns if the song is explicit or not
def getExplicit(songID):
    return sp.track(songID)['explicit']

In [9]:
#getTrackAttributes function that takes in a songID and returns song attributes such as, Tempo, Loudness, Key and Duration
def getTrackAttributes(songID):
    return sp.audio_features(tracks=songID)[0]

In [10]:
#def getReleaseDate takes a songID
#and uses the spotify API to allocate the release date of the song's album (assuming that album release date = song's release date)
def getReleaseDate(songID):
    return sp.track(songID)["album"]['release_date']

In [11]:
#getSongKey function that takes in a pitch class and returns the key. Reference: https://en.wikipedia.org/wiki/Pitch_class
def getSongKey(pitchClass):
    if pitchClass == 0:
        return 'C'
    elif pitchClass == 1:
        return 'C#, Db'
    elif pitchClass == 2:
        return 'D'
    elif pitchClass == 3:
        return 'D#, Eb'
    elif pitchClass == 4:
        return 'E'
    elif pitchClass == 5:
        return 'F'
    elif pitchClass == 6:
        return 'F#, Gb'
    elif pitchClass == 7:
        return 'G'
    elif pitchClass == 8:
        return 'G#, Ab'
    elif pitchClass == 9:
        return 'A'
    elif pitchClass == 10:
        return 'A# Bb'
    elif pitchClass == 11:
        return 'B'
    else: return None

In [1]:
#iteratte through songs in songs data frame (which we pulled from billboard.com) to grab song attributes
#using Spotify's API for things like, isExplicit, tempo, loudness, duration, release date.

for index, row in songs.iterrows():
    
    #get explicit attribute from Get Track API
    explicitValue = getExplicit(row['SpotifyID'])
    attributes = getTrackAttributes(row['SpotifyID'])
    
    #filling in values
    songs.isExplicit.iloc[index] = explicitValue
    songs.Tempo.iloc[index] = attributes['tempo']
    songs.Loudness.iloc[index] = attributes['loudness']
    songs.Duration.iloc[index] = attributes['duration_ms']/1000 #converting from milliseconds to seconds
    songs.Key.iloc[index] = getSongKey(attributes['key'])
    
    songs.ReleaseDate.iloc[index] = getReleaseDate(row['SpotifyID'])

In [14]:
songs.head()

,Title,Artist,SpotifyID,Tempo,Loudness,Key,isExplicit,Duration,ReleaseDate
0,Fine Line,Harry Styles,6VzcQuzTNTMFnJ6rBSaLH9,120.996,-8.5,D,False,377.96,2019-12-13
1,Hollywood's Bleeding,Post Malone,7sWRlDoTDX8geTR8zzr2vt,130.218,-3.206,E,False,156.267,2019-09-06
2,Lover,Taylor Swift,1dGr1c8CrMLDpV6mPbImSI,68.534,-7.582,G,False,221.307,2019-08-23
3,Rudolph The Red-Nosed Reindeer,Burl Ives,47otoIkhx3fkdivEXL5OB6,152.649,-12.755,C,False,130.027,1965-01-01
4,What You See Is What You Get,Luke Combs,273TiTHLlHSRe5zrzs7wvD,103.949,-4.892,F,False,171.92,2019-11-08


In [16]:
#songs.to_pickle('songsWithAttributes3.7.pkl')